In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

result_df = pd.read_csv('PaLM2input.csv', index_col=0)

In [2]:
result_df['GPT_input'] = result_df['GPT_input'].str.replace('minimum', 'min')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('maximum', 'max')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('average', 'avg')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('maxntprobnp', 'max ntprobnp')

In [3]:
result_df['GPT-Diagnoses'] = np.nan
result_df['GPT-Eval'] = np.nan

In [4]:
import vertexai

vertexai.init(project="n8n-copilot", location="us-central1")

In [5]:
# from vertexai.preview.language_models import TextGenerationModel
# parameters = {
#     "temperature": 0.2,
#     "max_output_tokens": 8192,
#     "top_p": 0.8,
#     "top_k": 40
# }
# model = TextGenerationModel.from_pretrained("medlm-medium")

In [6]:
from vertexai.preview.generative_models import GenerativeModel

multimodal_model = GenerativeModel("gemini-2.0-flash")

In [6]:
# config = {
#         "temperature": 0
#     }

In [7]:
res = multimodal_model.generate_content(
        ['Give me a plan for a 4-day holiday in Mayrhofen'])

In [8]:
print(res.candidates[0].content.parts[0].text)

## 4-Day Mayrhofen Adventure: A Plan for May

This plan focuses on experiencing the best of Mayrhofen, balancing outdoor activities with relaxation and local culture. It assumes you're arriving in Mayrhofen in the morning on Day 1 and departing in the late afternoon/evening on Day 4.

**Theme:** Active Adventure & Alpine Charm

**Accommodation:** This plan assumes you have pre-booked accommodation in Mayrhofen or a nearby village like Hippach or Finkenberg.

**Key Considerations:**

*   **Weather:** May can be unpredictable in the Alps. Pack layers, including waterproof clothing. Check the weather forecast daily.
*   **Cable Car Operations:** Check the operating status of the cable cars before planning your day, as some may have limited hours or be closed for maintenance in the shoulder season.
*   **Hiking Conditions:** Lower altitude hikes are generally good in May. Higher altitude trails might still have snow. Check trail reports before venturing out.
*   **Tourist Crowds:** May is 

In [9]:
prompt_no_example = '''You are an expert diagnostician machine for use by doctors. If the user input is not patient data, you politely decline the request. Please suggest diagnoses and conditions, followed by the evidence points supporting each diagnosis in the form of bullet points. Include previous diagnoses and pertinent information about the patient's medical history (if any). Pay close attention to all the history and investigations provided.  Put asterisks around the diagnoses to highlight them. Give each evidence points as a separate bullet point beneath the diagnosis. Include in your evidence points any relevant clinical scores that can be calculated from the information I have given. Do not explain the evidence points, only state them. For every diagnosis you list, if there are alternative differentials possible, state the most likely three in a bullet point beneath the evidence points (you do not need to state the evidence supporting them - you only need to do that for the main diagnoses). For the main diagnoses, give only confirmed diagnoses and evidence points that can be inferred solely based on the information I have given - do not use any other information. Only give me the information I have asked for - do not give me any other information. Do not give me any introductions or conclusions, safety instructions, or safety warnings. Use British English.

                                To illustrate how the information should be presented:

                                *MAIN DIAGNOSIS 1 AS HEADING*
                                evidence points to support MAIN DIAGNOSIS 1
                                The final bullet point is alternative differentials to consider: alternative 1, alternative 2, alternative 3

                                *MAIN DIAGNOSIS 2 AS HEADING*
                                evidence points to support MAIN DIAGNOSIS 2
                                The final bullet point is alternative differentials to consider: alternative 1, alternative 2, alternative 3

                                and so on...

Before finalising your answer check if you haven't missed any abnormal data points and hence any diagnoses or alternative differentials that could be made based on them. If you did, add them to your reply. If two diagnoses are commonly caused by the same underlying disease, have them under one header, which is the underlying disease.
'''

example = ''''''

patient_data = '''

Patient data:\n'''

In [10]:
repeat = []
for i in tqdm(range(0,1000)):
    try:
        response = multimodal_model.generate_content(
            [prompt_no_example + example + patient_data + result_df.iloc[i]['GPT_input']]
        )
        hadm_id = result_df.index[i]
        result_df.loc[hadm_id, 'GPT-Diagnoses'] = response.candidates[0].content.parts[0].text
    except Exception as e: 
        print('Error happened at iteration i: ' + str(i))
        repeat.append(i)
        print(e)

 24%|██▍       | 241/1000 [10:26<33:25,  2.64s/it]  

Error happened at iteration i: 240
list index out of range


100%|██████████| 1000/1000 [42:28<00:00,  2.55s/it]


In [11]:
for i in tqdm(repeat):
    try:
        response = multimodal_model.generate_content(
            [prompt_no_example + example + patient_data + result_df.iloc[i]['GPT_input']], generation_config=config
        )
        hadm_id = result_df.index[i]
        result_df.loc[hadm_id, 'GPT-Diagnoses'] = response.candidates[0].content.parts[0].text
    except Exception as e: 
        print('Error happened at iteration i: ' + str(i))
        print(e)

100%|██████████| 1/1 [00:00<00:00, 2882.68it/s]

Error happened at iteration i: 240
name 'config' is not defined


In [13]:
result_df['GPT-Diagnoses'].isna().sum()

1

In [14]:
hadm_id = result_df.index[i]
result_df.loc[hadm_id, 'GPT-Diagnoses'] = 'Model declined to answer due to recitation'

In [15]:
result_df.to_csv('TempdefGemini20Prod.csv')